In [ ]:
# load data
import json

file_path="" # 자신이 학습 시킨 데이터 추가하기
with open(file_path, "r") as f:
    crm_lines = f.readlines()

In [ ]:
initial_response_path="" # estimate_initial_answer.py 로 돌린 결과 file path 넣기
with open(initial_response_path, "r") as f:
    verdicts = f.readlines()

In [ ]:
# 각 문제에 대해서 initial response가 맞추었는지, 틀렸는지 dictionary에 저장
# - idx: True/False
verdict_dict={}
for verdict in verdicts:
    verdict = json.loads(verdict)
    verdict_dict[verdict['idx']] = verdict['correct']

In [ ]:
# 각 index에 대해서 feedback 모으기
# - data[idx] = [inst1, inst2, inst3]
import json
from collections import defaultdict

data = defaultdict(list)

for line in crm_lines:
    inst = json.loads(line)
    idx = inst['idx']
    if idx not in data:
        data[idx] = []
    data[idx].append(inst)

# Generate pairwise data

In [ ]:
pairwise_data=[]
correct_pairwise_data=[]
wrong_pairwise_data=[]

def cmp_func(idx, question, initial_response, initial_verdict, inst1, inst2):
    score1 = inst1['score']
    score2 = inst2['score']
    if score1 == score2:
        return None
    elif score1 > score2:
        return {
            'idx': idx,
            'question': question,
            'initial_response': initial_response,
            'initial_verdict': initial_verdict,
            'feedback_w': inst1['feedback'],
            'feedback_l': inst2['feedback'],
            'w_score': score1,
            'l_score': score2,
        }
    else:
        return {
            'idx': idx,
            'question': question,
            'initial_response': initial_response,
            'initial_verdict': initial_verdict,
            'feedback_w': inst2['feedback'],
            'feedback_l': inst1['feedback'],
            'w_score': score2,
            'l_score': score1,
        }
    

for key in data:
    question, initial_response = data[key][0]['question'], data[key][0]['initial_response']
    initial_verdict = verdict_dict[key]
    inst1, inst2, inst3 = data[key][0], data[key][1], data[key][2]

    new_pair = cmp_func(key, question, initial_response, initial_verdict, inst1, inst2)
    if new_pair is not None:
        pairwise_data.append(new_pair)
    # cmp 2,3
    new_pair = cmp_func(key, question, initial_response, initial_verdict, inst2, inst3)
    if new_pair is not None:
        pairwise_data.append(new_pair)
    # cmp 1,3
    new_pair = cmp_func(key, question, initial_response, initial_verdict, inst1, inst3)
    if new_pair is not None:
        pairwise_data.append(new_pair)